# Import libraries

In [1]:
from src.preprocessing.dataLoader_CelebA import get_partitioned_dataloaders, create_subset_loader
from src.ml.resNet50 import SiameseResNet
import torch
from src.ml.hyperparam_study import run_optuna_study

# Import losses

In [2]:
from pytorch_metric_learning.losses import ContrastiveLoss
from pytorch_metric_learning.losses import MarginLoss
from pytorch_metric_learning.losses import AngularLoss
from pytorch_metric_learning.losses import CosFaceLoss
from pytorch_metric_learning.losses import MultiSimilarityLoss
from pytorch_metric_learning.losses import HistogramLoss

# 1. Load the data

In [3]:
IMAGE_DIR = "data/celeba/img_align_celeba"
LABEL_FILE = "data/celeba/identity_CelebA.txt"
PARTITION_FILE = "data/celeba/list_eval_partition.csv"
IMG_SIZE = 224
BATCH_SIZE = 32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
train_loader, val_loader, test_loader = get_partitioned_dataloaders(image_dir= IMAGE_DIR,
                                                               label_file= LABEL_FILE,
                                                               partition_file= PARTITION_FILE,
                                                               batch_size=BATCH_SIZE,
                                                               img_size=IMG_SIZE)

# Create the model

In [5]:
model = SiameseResNet()

# Find best Hyperparameters

# Train Model

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-5, weight_decay=1e-4)

In [7]:
contrastive_loss = ContrastiveLoss(neg_margin=1.0, pos_margin=0)
histogram_loss = HistogramLoss(n_bins=1000)
multi_similarity_loss = MultiSimilarityLoss()
margin_loss = MarginLoss(margin=0.5)

In [8]:
results = model.train_model(
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=histogram_loss,
        optimizer=optimizer,
        num_epochs=10,
        device=DEVICE,
        patience=5,
        experiment_name='SiameseResNet',
        tuning_mode=False
    )

Epoch 1/10:   1%|          | 34/5086 [03:39<9:04:11,  6.46s/it, batch_loss=0.24866499]


RuntimeError: index 1001 is out of bounds for dimension 0 with size 1001

In [ ]:
train_loader_study = (create_subset_loader(train_loader,10000))
val_loader_study = (create_subset_loader(train_loader,2000))
study = run_optuna_study(train_loader_study, val_loader_study, n_trials=10, study_name="siamese_constrastive_HP_study")
best_params = study.best_params

# Plot the results

In [ ]:
! mlflow ui --port 5000